<a href="https://colab.research.google.com/github/0xLighty/Natural-Language-Processing/blob/main/hw4_tast2_nlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Task 2
Import BERT for classification model which is already fine tuned and apply it on any text classification dataset such as the twitter dataset

In [1]:
!pip install transformers
!pip install datasets
!pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from sklearn.model_selection import train_test_split
from datasets import load_dataset
from transformers import get_scheduler
import evaluate
from tqdm import tqdm
from torch.utils.data import DataLoader
import pandas as pd
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline

In [3]:
# train = load_dataset('tweet_eval', 'emotion', split='train[:2000]')
val = load_dataset('tweet_eval', 'emotion', split='validation[:200]')

In [4]:
type(val)

datasets.arrow_dataset.Dataset

In [5]:
val

Dataset({
    features: ['text', 'label'],
    num_rows: 200
})

In [6]:
val['text'][:10]

['@user @user Oh, hidden revenge and anger...I rememberthe time,she rebutted you.',
 'if not then #teamchristine bc all tana has done is provoke her by tweeting shady shit and trying to be a hard bitch begging for a fight',
 'Hey @user #Fields in #skibbereen give your online delivery service a horrible name. 1.5 hours late on the 1 hour delivery window.',
 'Why have #Emmerdale had to rob #robron of having their first child together for that vile woman/cheating sl smh #bitter',
 '@user I would like to hear a podcast of you going off refuting her entire article. Extra indignation please.',
 "If I have to hear one more time how I am intimidate men... I'm going to explode! Why are guys these days so pussified?",
 'depression sucks😔',
 'O, the melancholy Catacombs quickly wandered about the Rue Morgue, Madman!',
 '#RIPBiwott I think Robert oukos soul can now rejoice and rest in peace. Call an evil man evil and a good man a good man. He was an evil man',
 '@user @user *on his back. Apologies

In [7]:
val['label'][:10]

[0, 0, 0, 0, 0, 0, 3, 3, 0, 3]

In [8]:
# there are four classes/labels/emotions
set(val['label'])

{0, 1, 2, 3}

In [9]:
# from transformers import BertForQuestionAnswering, BertForSequenceClassification,BertTokenizer
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline #, AdamW, BertTokenizerFast, Trainer, TrainingArguments

In [10]:
tokenizer = AutoTokenizer.from_pretrained('philschmid/BERT-tweet-eval-emotion')
model = AutoModelForSequenceClassification.from_pretrained('philschmid/BERT-tweet-eval-emotion')
classifier = pipeline('text-classification', tokenizer=tokenizer, model=model)

In [11]:
# taking random text and it's relevant class/emotion/label
random_text = []
random_num = np.random.randint(0,len(val))
random_text.append(val['text'][random_num])
cls_label = val['label'][random_num]

In [12]:
# encoding random text with max_length of 512
input_ids = tokenizer.encode(random_text,is_split_into_words=True, max_length=512,truncation=True)
print("The input has a total of {} tokens.".format(len(input_ids)))

The input has a total of 20 tokens.


In [13]:
# visualizing the tokens
tokens = tokenizer.convert_ids_to_tokens(input_ids)
for token, id in zip(tokens, input_ids):
    print('{:8}{:8,}'.format(token,id))

[CLS]        101
@            137
user       4,795
@            137
user       4,795
[UNK]        100
snow       4,883
##f        2,087
##lake    13,238
random     7,091
such       1,216
a            170
funny      6,276
man        1,299
never      1,309
a            170
dull      10,884
moment     1,721
brilliant   8,431
[SEP]        102


In [14]:
# Visualizing the number of token in text
sep_token_idx = input_ids.index(tokenizer.sep_token_id)
print("SEP token index: ", sep_token_idx)

SEP token index:  19


In [15]:
num_seg_a = sep_token_idx + 1
print("Number of tokens in text: ", num_seg_a)

Number of tokens in text:  20


In [16]:
#creating the segment ids and making sure every input token has a segment id
segment_ids = [0]*num_seg_a 
assert len(segment_ids) == len(input_ids)

In [17]:
output = model(torch.tensor([input_ids]),  token_type_ids = torch.tensor([segment_ids]))

In [18]:
pred = int(torch.argmax(output.logits))

In [19]:
print("Predicted class : {} - Original Class : {}".format(pred, cls_label))

Predicted class : 1 - Original Class : 1
